In [1]:
#UCB Project 1 Homelessness vs real estate prices
#Read and clean up Zillow Economics Data (1996-2017) 
#CITY DATA

In [2]:
#Import dependencies
import csv
import pandas as pd
from pandas import *
from IPython.display import display

In [3]:
#Read and assess the file
data_chunks = read_csv("City_time_series.csv", iterator=True, chunksize=500000)
city_prices_df= concat(data_chunks, ignore_index=True)
display(city_prices_df.head())

city_geo_df= read_csv("cities_crosswalk.csv")
city_geo_df= city_geo_df.rename(columns = {'Unique_City_ID' : 'RegionName'})
display(city_geo_df.head())

,Date,RegionName,HomesSoldAsForeclosuresRatio_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,abbottstownadamspa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,108700.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,aberdeenharfordmd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81300.0,137900.0,109600.0,108600.0,147900.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,aberdeenmonroems,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,44700.0,44700.0,74500.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,aberdeenmoorenc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,87400.0,131000.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,abilenetaylortx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,RegionName,City,County,State
0,oak_grovechristianky,Oak Grove,Christian,KY
1,jarvisburgcurritucknc,Jarvisburg,Currituck,NC
2,mcminnvilleyamhillor,McMinnville,Yamhill,OR
3,union_townshiperiepa,Union Township,Erie,PA
4,oshkoshwinnebagowi,Oshkosh,Winnebago,WI


In [4]:
#Output anatomy of files
print ('Total # of rows in df: %s' % (len(city_prices_df)))
print ('Total # of columns in df: %s' % len(city_prices_df.columns))
#df.info()
#print(list(city_prices_df))

print ('Total # of rows in region_df: %s' % (len(city_geo_df)))
print ('Total # of columns in region_df: %s' % len(city_geo_df.columns))
#df.info()
#print(list(city_geo_df))

Total # of rows in df: 4172721
Total # of columns in df: 85
Total # of rows in region_df: 25341
Total # of columns in region_df: 4


In [5]:
#Create a new city_prices_df with only the columns of interest
# HVI - a smoothed seasonally adjusted measure of the median estimated home value across a given region and housing type
Zillow_df = city_prices_df[["Date","RegionName","MedianSoldPricePerSqft_AllHomes","MedianSoldPricePerSqft_SingleFamilyResidence",
               "MedianSoldPrice_AllHomes","ZHVIPerSqft_AllHomes","ZHVI_AllHomes", "ZHVI_SingleFamilyResidence"]]  
                            
#display(Zillow_df)

#Merge the two dataframes
merged_df = pd.merge(city_geo_df, Zillow_df, on="RegionName")
display(merged_df)

#Checkout merged_df output
print ('Total # of rows in df: %s' % (len(merged_df)))
print ('Total # of columns in df: %s' % len(merged_df.columns))
#merged_df.info()

,RegionName,City,County,State,Date,MedianSoldPricePerSqft_AllHomes,MedianSoldPricePerSqft_SingleFamilyResidence,MedianSoldPrice_AllHomes,ZHVIPerSqft_AllHomes,ZHVI_AllHomes,ZHVI_SingleFamilyResidence
0,oak_grovechristianky,Oak Grove,Christian,KY,1996-04-30,NaN,NaN,NaN,62.0,70000.0,70000.0
1,oak_grovechristianky,Oak Grove,Christian,KY,1996-05-31,NaN,NaN,NaN,62.0,69900.0,69900.0
2,oak_grovechristianky,Oak Grove,Christian,KY,1996-06-30,NaN,NaN,NaN,62.0,69700.0,69700.0
3,oak_grovechristianky,Oak Grove,Christian,KY,1996-07-31,NaN,NaN,NaN,62.0,70000.0,70000.0
4,oak_grovechristianky,Oak Grove,Christian,KY,1996-08-31,NaN,NaN,NaN,62.0,70100.0,70100.0
5,oak_grovechristianky,Oak Grove,Christian,KY,1996-09-30,NaN,NaN,NaN,62.0,70000.0,70000.0
6,oak_grovechristianky,Oak Grove,Christian,KY,1996-10-31,NaN,NaN,NaN,61.0,69900.0,69900.0
7,oak_grovechristianky,Oak Grove,Christian,KY,1996-11-30,NaN,NaN,NaN,61.0,69700.0,69700.0
8,oak_grovechristianky,Oak Grove,Christian,KY,1996-12-31,NaN,NaN,NaN,61.0,69100.0,69100.0
9,oak_grovechristianky,Oak Grove,Christian,KY,1997-01-31,NaN,NaN,NaN,61.0,68400.0,68400.0


Total # of rows in df: 4172721
Total # of columns in df: 11


In [37]:
#Clean up dataset
#Set date as index
mod_merged_df=merged_df.set_index('Date')
#display (mod_merged_df)

#Remove data prior to 2007
ten_year_merged_df= mod_merged_df.sort_index().truncate(before='2007-01-31')
display (ten_year_merged_df)
#ten_year_merged_df.info()

,RegionName,City,County,State,MedianSoldPricePerSqft_AllHomes,MedianSoldPricePerSqft_SingleFamilyResidence,MedianSoldPrice_AllHomes,ZHVIPerSqft_AllHomes,ZHVI_AllHomes,ZHVI_SingleFamilyResidence
Date,,,,,,,,,,
2007-01-31,weber_cityscottva,Weber City,Scott,VA,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-31,cascocumberlandme,Casco,Cumberland,ME,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-31,shelby_townshipmacombmi,Shelby Township,Macomb,MI,NaN,NaN,NaN,126.0,224100.0,239500.0
2007-01-31,monroevilleallenin,Monroeville,Allen,IN,NaN,NaN,NaN,69.0,108600.0,108600.0
2007-01-31,blanchardcentrepa,Blanchard,Centre,PA,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-31,edwardsvillemadisonil,Edwardsville,Madison,IL,NaN,NaN,NaN,102.0,170900.0,171700.0
2007-01-31,stoughtondanewi,Stoughton,Dane,WI,NaN,NaN,176345.0,104.0,179600.0,181700.0
2007-01-31,atlantaloganil,Atlanta,Logan,IL,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-31,concordmiddlesexma,Concord,Middlesex,MA,NaN,NaN,NaN,339.0,696200.0,749500.0


In [39]:
ten_year_merged_df=ten_year_merged_df.reset_index().set_index('City') 
# grouped_city_state= ten_year_merged_df.groupby(["City"])
# grouped_city_state.mean()
#grouped_city_state = ten_year_merged_df.groupby(['City','State']['RegionName']).apply(','.join).reset_index() #this is 2nd option
display(ten_year_merged_df)

,Date,RegionName,County,State,MedianSoldPricePerSqft_AllHomes,MedianSoldPricePerSqft_SingleFamilyResidence,MedianSoldPrice_AllHomes,ZHVIPerSqft_AllHomes,ZHVI_AllHomes,ZHVI_SingleFamilyResidence
City,,,,,,,,,,
Weber City,2007-01-31,weber_cityscottva,Scott,VA,NaN,NaN,NaN,NaN,NaN,NaN
Casco,2007-01-31,cascocumberlandme,Cumberland,ME,NaN,NaN,NaN,NaN,NaN,NaN
Shelby Township,2007-01-31,shelby_townshipmacombmi,Macomb,MI,NaN,NaN,NaN,126.0,224100.0,239500.0
Monroeville,2007-01-31,monroevilleallenin,Allen,IN,NaN,NaN,NaN,69.0,108600.0,108600.0
Blanchard,2007-01-31,blanchardcentrepa,Centre,PA,NaN,NaN,NaN,NaN,NaN,NaN
Edwardsville,2007-01-31,edwardsvillemadisonil,Madison,IL,NaN,NaN,NaN,102.0,170900.0,171700.0
Stoughton,2007-01-31,stoughtondanewi,Dane,WI,NaN,NaN,176345.0,104.0,179600.0,181700.0
Atlanta,2007-01-31,atlantaloganil,Logan,IL,NaN,NaN,NaN,NaN,NaN,NaN
Concord,2007-01-31,concordmiddlesexma,Middlesex,MA,NaN,NaN,NaN,339.0,696200.0,749500.0


In [44]:
#Selected cities below are based on cross-examination of data available for cities in HUD dataset
Cities = ["New York", "Philadelphia", "Boston", "Washington", "Chicago", "Minneapolis",
         "Denver", "Salt Lake City", "Seattle", "Los Angeles", "San Francisco", "Miami", 
         "Charlotte", "Atlanta", "Detroit", "Anchorage", "Honolulu", "Indianapolis"]

Selected_cities= (ten_year_merged_df.loc[Cities])
#Print csv file
Selected_cities.to_csv("selected_citites.csv")
display(Selected_cities)

#-------THE ABOVE CODE WORKS BUT I NEED TO USE STATE AS WELL----CONTINUE WORKING ON THIS

# Region=[]
# Cities=['Atlanta']
# States=['GA']

# for row in city_geo_df:
#     if city in Cities and state in States:
#         Region.append([RegionName])

# print(Region)



# #Create a new dataframe only with data for the selected cities -WORK IN PROGRESS
# for row in merged_df:
#     if city in Cities:
#         df.loc['City', 'City':'ZHVI_SingleFamilyResidence']


#Further Clean dataframe - remove data before 2007 and remove columns mostly empty
# df.dropna(axis=1, how='any', inplace=True)
# display(df)
# print ('Total # of rows: %s' % (len(Zillow_df)))
# print ('Total # of columns: %s' % len(Zillow_df.columns))

,Date,RegionName,County,State,MedianSoldPricePerSqft_AllHomes,MedianSoldPricePerSqft_SingleFamilyResidence,MedianSoldPrice_AllHomes,ZHVIPerSqft_AllHomes,ZHVI_AllHomes,ZHVI_SingleFamilyResidence
City,,,,,,,,,,
New York,2007-01-31,new_yorkqueensny,Queens,NY,386.8288,325.8641,477220.00,386.0,518800.0,532600.0
New York,2007-02-28,new_yorkqueensny,Queens,NY,392.0199,324.3134,473579.00,385.0,515800.0,530200.0
New York,2007-03-31,new_yorkqueensny,Queens,NY,401.2203,322.0690,478744.00,385.0,516200.0,528200.0
New York,2007-04-30,new_yorkqueensny,Queens,NY,419.6610,328.8118,495497.10,385.0,516200.0,526300.0
New York,2007-05-31,new_yorkqueensny,Queens,NY,449.5610,329.5494,515855.90,383.0,515500.0,524600.0
New York,2007-06-30,new_yorkqueensny,Queens,NY,452.9427,333.8735,531150.00,380.0,512200.0,522400.0
New York,2007-07-31,new_yorkqueensny,Queens,NY,476.1741,330.8305,550055.00,378.0,509300.0,520200.0
New York,2007-08-31,new_yorkqueensny,Queens,NY,469.8995,333.8789,552038.50,378.0,510100.0,518500.0
New York,2007-09-30,new_yorkqueensny,Queens,NY,455.3382,334.4008,543741.50,378.0,511800.0,517200.0
